# Examining Racial Discrimination in the US Job Market

### Background
Racial discrimination continues to be pervasive in cultures throughout the world. Researchers examined the level of racial discrimination in the United States labor market by randomly assigning identical résumés to black-sounding or white-sounding names and observing the impact on requests for interviews from employers.

### Data
In the dataset provided, each row represents a resume. The 'race' column has two values, 'b' and 'w', indicating black-sounding and white-sounding. The column 'call' has two values, 1 and 0, indicating whether the resume received a call from employers or not.

Note that the 'b' and 'w' values in race are assigned randomly to the resumes when presented to the employer.

### Exercises
You will perform a statistical analysis to establish whether race has a significant impact on the rate of callbacks for resumes.

Answer the following questions **in this notebook below and submit to your Github account**. 

   1. What test is appropriate for this problem? Does CLT apply?
   2. What are the null and alternate hypotheses?
   3. Compute margin of error, confidence interval, and p-value.
   4. Write a story describing the statistical significance in the context or the original problem.
   5. Does your analysis mean that race/name is the most important factor in callback success? Why or why not? If not, how would you amend your analysis?

You can include written notes in notebook cells using Markdown: 
   - In the control panel at the top, choose Cell > Cell Type > Markdown
   - Markdown syntax: http://nestacms.com/docs/creating-content/markdown-cheat-sheet


#### Resources
+ Experiment information and data source: http://www.povertyactionlab.org/evaluation/discrimination-job-market-united-states
+ Scipy statistical methods: http://docs.scipy.org/doc/scipy/reference/stats.html 
+ Markdown syntax: http://nestacms.com/docs/creating-content/markdown-cheat-sheet
****

In [13]:
import pandas as pd
import numpy as np
from scipy import stats

In [14]:
data = pd.io.stata.read_stata('data/us_job_market_discrimination.dta')

In [15]:
# number of callbacks for black-sounding names
black_callbacks = sum(data[data.race=='b'].call)
print("There were", black_callbacks, "callbacks for black-sounding names")

There were 157.0 callbacks for black-sounding names


In [16]:
# total number of black-sounding names 
total_black_names = len(data[data.race=='b'])
print("There were", total_black_names, "total black-sounding names")

There were 2435 total black-sounding names


In [17]:
# sample proportion of callbacks for black-sounding names
sample_prop_black = black_callbacks / total_black_names
print("The observed proportion of callbacks for black-sounding names is", sample_prop_black, "(~6.45%)")

The observed proportion of callbacks for black-sounding names is 0.064476386037 (~6.45%)


In [18]:
# number of callbacks for white-sounding names
white_callbacks = sum(data[data.race=='w'].call)
print("There were", white_callbacks, "callbacks for white-sounding resumes")

There were 235.0 callbacks for white-sounding resumes


In [19]:
# total number of white-sounding names 
total_white_names = len(data[data.race=='w'])
print("There were", total_white_names, "total white-sounding names")

There were 2435 total white-sounding names


In [22]:
# sample proportion of callbacks for white-sounding names
sample_prop_white = white_callbacks / total_white_names
print("The observed proportion of callbacks for white-sounding names is", sample_prop_white, "(~9.65%)")

The observed proportion of callbacks for white-sounding names is 0.0965092402464 (~9.65%)


# 1. What test is appropriate and does the Central Limit Theorem apply?

<b>Test:</b> In the present case, we are testing whether there is a significant difference between the proportion of résumés with black-sounding names receiving callbacks versus the proportion of résumés with white-sounding names.  Because we testing the significance of the difference between two separate populations, a two-sample test is appropriate.  Furthermore, while we don't know the true population standard deviation for résumés with black-sounding names or for résumés with white sounding names, the samples are both sufficiently large ($n=2435$) to warrant the use of a $z-statistic$ in this analysis.

<b>Central Limit Theorem:</b> In order to apply the Central Limit Theorem ("CLT"), there are several assumptions and conditions that must be satisfied:
<ol>
<li><b>Randomness:</b> the data must be randomly sampled
<li><b>Independence:</b> the sample values must be independent of eachother
<li><b>10% Condition:</b> when sampled without replacement, the sample size, $n$, must be less than 10% of the population
<li><b>Sample Size:</b> the sample size must be sufficiently large (rule of thumb: $n\geq30$)
</ol>

In the present case, we are told that the researchers randomly assigned identical résumés to black-sounding or white-sounding names and observed the results.  Therefore, the randomness condition appears to be satisfied.

Assuming résumé selection was truly random, there is no basis for the selection of one résumé for assignment to a black or white-sounding name to influence the selection and assignment of the next résumé.  Therefore, the independence condition appears to be satisfied.

Our sample size for both samples (résumés with black-sounding names and résumés with white-sounding names) is 2435.  If we assume that the relevant population in each case is the <i>total number of résumés</i> with black-sounding names and white-sounding names even in the U.S. alone, we can see that our sample size is clearly less than 10% of the total population of résumés in existence.  Thus, the 10% rule is satisfied.

Here, each sample size, $n_b$ and $n_w$ is equal to 2435 and therefore large enough to satisfy the size requirement of the CLT.

<b>Conclusion:</b> All of the conditions for the application of the CLT are satisfied by our sample data.

# 2. What are the null and alternate hypotheses?

<b>Null Hypothesis ($H_0$):</b> there is no difference between the success rate (i.e., % of callbacks) for résumés with black-sounding names versus those with white-sounding names.

<b>Alternative Hypothesis ($H_1$):</b> résumés with a black-sounding name experience a significant difference in success rate when compared with those with white-sounding names.

# 3. Compute margin of error, confidence interval, and p-value.

In order to calculate the margin of error for the true difference in proportion of callbacks for résumés with black-so names versus résumés with white sounding names ($\mu_{\bar{p}_1-\bar{p}_2})$, we need to calculate the combined variance and standard deviation from the two samples:

$$\sigma^2_{\bar{p}_1-\bar{p}_2}=\frac{P_1(1-P_1)}{n_1}+\frac{P_2(1-P_2)}{n_2}$$

We don't know the true population proportions $P_1$ and $P_2$ but we can approximate using the observed sample propotions $\bar{p}_1$ and $\bar{p}_2$ and the unbiased denominator of $n-1$.  The formulas is as follows:

$$\sigma^2_{\bar{p}_1-\bar{p}_2}=\frac{\bar{p}_1(1-\bar{p}_1)}{n_1-1}+\frac{\bar{p}_2(1-\bar{p}_2)}{n_2-1}$$

Therefore, in order to calculate the standard error of our sampling distribution of the difference in sample proportions $\bar{p}_1$ and $\bar{p}_2$, the fomula is as follows:

$$\sigma_{\bar{p}_1-\bar{p}_2}=\sqrt{\frac{\bar{p}_1(1-\bar{p}_1)}{n_1-1}+\frac{\bar{p}_2(1-\bar{p}_2)}{n_2-1}}$$

To calculate a 95% confidence interval of the true difference in population proportions $P_1$ and $P_2$ we rely on the CLT and normal distribution which tell us that 95% of the values of a normal distribution are within 1.96 standard errors (i.e., +/-$1.96(\sigma_{\bar{p}_1-\bar{p}_2})$) of the mean.  Stated another way, when calculated using the methods outlined above, the resulting interval will contain the true difference in population proportion 95% of the time.

In [29]:
# compute standard error of sampling distribution
diff_prop_std_error = np.sqrt(sample_prop_black*(1-sample_prop_black)/(total_black_names - 1) + 
                                    sample_prop_white*(1-sample_prop_white)/(total_white_names - 1))

print("Standard error of the sampling distribution (combined samples):", diff_prop_std_error)

Standard error of the sampling distribution (combined samples): 0.00778496930716


In [30]:
observed_diff_proportions = sample_prop_white - sample_prop_black

# compute margin of error and 95% confidence interval
diff_prop_margin_error = 1.96 * diff_prop_std_error

conf_interval_low_end = observed_diff_proportions - diff_prop_margin_error
conf_interval_high_end = observed_diff_proportions + diff_prop_margin_error

print("The observed difference in sample proportions is", observed_diff_proportions)
print("The margin of error is", diff_prop_margin_error)
print("The 95% confidence interval is", conf_interval_low_end, "to", conf_interval_high_end)

The observed difference in sample proportions is 0.0320328542094
The margin of error is 0.015258539842
The 95% confidence interval is 0.0167743143674 to 0.0472913940515


<b>Confidence Interval:</b> we are 95% confident that the true difference in the population proportion of callbacks for résumés with black-sounding names and the population proportion of résumés with white-sounding names is between ~1.68% and ~4.73%.

<b>Hypothesis Test:</b> we can test the above-stated null hypothesis ($H_0$) by comparing our observed difference in sample proportions to the expected difference if $N_0$ were true (i.e., 0%).  To do so, we combine both samples and compute a single "success" (i.e., callback) proportion for the combined sample.  We then plug that combined sample proportion into the formula above to calculate the standard error of our theoretical sampling distribution, assuming $H_0$ is true.  Because $H_0$ states that there is no difference in the true population proportions, the $\mu$ of our theoretical sampling distribution will be 0.  We then find a z-score (i.e., how many standard errors our observed difference in sample proportions is from the theoretical mean) and use it to calculate the a p-value of our observation.

Given that our theoretical variance and standard deviation are being calculated with a single $P$ value (calcualted from the combined samples), the standard error formula for the theoretical combined distribution above can be simplified as follows:

$$\sigma=\sqrt{\frac{2p_c(1-p_c)}{n_c-1}}$$

Where $p_c$ is the combined sample proportion and $n_c$ is the combined sample size.

In [23]:
# compute proportion of successes (callbacks) from the combined samples as a single sample proportion
total_callbacks = black_callbacks + white_callbacks
combined_sample_size = total_black_names + total_white_names
total_callback_proportion = total_callbacks / combined_sample_size

print("Combined sample callback proportion:", total_callback_proportion)

Combined sample callback proportion: 0.0804928131417


In [32]:
# compute standard error of theoretical distribution
combined_dist_std_error = np.sqrt(2*total_callback_proportion*(1-total_callback_proportion)/(4870-1))

print("Theoretical combined sampling distribution standard error:", combined_dist_std_error)

Theoretical combined sampling distribution standard error: 0.00551380277308


In [34]:
# compute z-score and p-value
z_score = observed_diff_proportions/combined_dist_std_error

one_tailed_p_value = stats.norm.sf(z_score) 
two_tailed_p_value = stats.norm.sf(z_score) * 2

print("The z-score for our observed difference in sample proportions is", z_score)
print("One tailed p-value:", one_tailed_p_value)
print("Two tailed p-value:", two_tailed_p_value)

The z-score for our observed difference in sample proportions is 5.80957562825
One tailed p-value: 3.1315698393e-09
Two tailed p-value: 6.26313967861e-09


<b>Conclusion:</b> assuming $H_0$ is true, the probability of observing a difference in our sample proportions <i>as extreme as</i> our observed difference is approximately 0.0000000063 and the probability of observing a difference in our sample proportions <i>as large as</i> our observed difference is approximately 0.0000000031.  These are very low probabilities and therefore we can reject $H_0$ in favor of $H_1$ stated above.

This does not allow us to conclude that race/name is the most important factor in determining whether a résumé will receive a callback.  The results of our analysis only support a conclusion that a statistically significant difference exists, and nothing more.  We would need to repeat this type of analysis controlling for other variables that are available in the data set in order to determine which factors are the most influential regarding likelihood of receiving a callback.  However, even after such an analysis it is important to note that our ability to make inferences and sweeping conclusions about success rate will be limited to the data we have available.  Common sense tells us that there are an infinite number of variables that can influence a decision to give an applicant a callback and it is virtually impossible to capture all of those variables for this type of analysis.  